# Docker-compose manipulation

In [11]:
#Build images
!docker-compose build --no-cache


traefik uses an image, skipping
mongo-express uses an image, skipping
mongo-express_userDB uses an image, skipping
Building mongo_manager_db_service
[+] Building 0.0s (0/1)                                          docker:default
[+] Building 0.2s (1/2)                                          docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 265B                                       0.0s
 => [internal] load metadata for docker.io/library/mongo:6.0               0.2s
[+] Building 0.3s (1/2)                                          docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 265B                                       0.0s
 => [internal] load metadata for docker.io/library/mongo:6.0               0.3s
[+] Building 0.5s (1/2)                                          docker:default
 => [internal] load build definition from Dockerfil

In [12]:
#Run containers
!docker-compose -f ./docker-compose.yml up -d

Creating network "parkman_app-network" with driver "bridge"
Creating parkman_mongo_user_db_service_1 ... 
Creating parkman_traefik_1               ... 
Creating mobile_app                      ... 
Creating parkman_mongo_manager_db_service_1 ... 
ting parkman_traefik_1                  ... doneCreating manager_app                        ... 
Creating mexpress_userDB                    ... 
Creating mexpress                           ... 
Creating parkman_nginx_1                    ... mdone
ting parkman_nginx_1                    ... done

In [13]:
#Stop containers
!docker-compose -f ./docker-compose.yml down

Stopping parkman_nginx_1                    ... 
Stopping mexpress                           ... 
Stopping mexpress_userDB                    ... 
Stopping manager_app                        ... 
Stopping parkman_mongo_manager_db_service_1 ... 
Stopping parkman_mongo_user_db_service_1    ... 
Stopping parkman_traefik_1                  ... 
Stopping mobile_app                         ... 
ping parkman_traefik_1                  ... doneRemoving parkman_nginx_1                    ... 
Removing mexpress                           ... 
Removing mexpress_userDB                    ... 
Removing manager_app                        ... 
Removing parkman_mongo_manager_db_service_1 ... 
Removing parkman_mongo_user_db_service_1    ... 
Removing parkman_traefik_1                  ... 
Removing mobile_app                         ... 
Removing network parkman_app-network    ... done


# Main setup notebook for docker compose
This segment exposes cells for defining docker-compose structure

## Docker-compose config

The cell below creates an empty docker-compose file. And clears out the .env of docker compose. <span style="color:red">**WARNING**</span> - running this cell will overwrite current docker compose. It should be run first when beginning config.

In [24]:
%%writefile docker-compose.yml
version: '3'

services:



networks:
  app-network:
    driver: bridge  # Use the default bridge network driver

volumes:

Overwriting docker-compose.yml


In [ ]:
%% writefile .env

## Creating ManagerApp and MobileApp services

Define requirements required for source code to successfully work in container

### ManagerAPP section

In [25]:
%%writefile ./ManagerApp/requirements.txt

Flask

Overwriting ./ManagerApp/requirements.txt


In [26]:
%%writefile ./ManagerApp/app.dockerfile

FROM python:3.12

# Set the working directory inside the container to /app
WORKDIR /app

# Copy the requirements.txt file into the container at /app
COPY requirements.txt .

# Install the Python dependencies specified in requirements.txt
RUN pip install --no-cache-dir -r requirements.txt

# Copy the application code into the container at /app
COPY ./source_code/ ./source_code/

# Expose port 80 to allow external access to the container's port 80
EXPOSE 80

# Specify the command to run when the container starts
CMD ["python", "./source_code/API_voditelj/app.py"]

Overwriting ./ManagerApp/app.dockerfile


The cell below adds necessary config to define ManagerApp service 

In [27]:
 
from ruamel.yaml import YAML

# Create a YAML object
yaml = YAML()
yaml.preserve_quotes = True  # Preserves quotes in the YAML file
yaml.indent(mapping = 2, sequence = 2, offset = 2)

#Setup file edit path 
docker_compose_path = './docker-compose.yml'  

# Read the docker-compose.yml file
with open(docker_compose_path, 'r') as file:
    docker_compose = yaml.load(file)

if docker_compose['services'] is None:
    docker_compose['services']={}

if docker_compose['volumes'] is None:
    docker_compose['volumes']={}

ManagerApp = {
    'container_name': 'manager_app', #TODO: important for nginx detection
    'build': {
        'context': './ManagerApp',
        'dockerfile': 'app.dockerfile'
    },
    'networks': [
        'app-network'
    ],



}

#Add the ManagerApp service to the docker-compose.yml file
docker_compose['services']['manager_app'] = ManagerApp

# Write the updated configuration back to docker-compose.yml
with open(docker_compose_path, 'w') as file:
    yaml.dump(docker_compose, file)

print("\ndocker-compose.yml has been updated successfully.")
 


docker-compose.yml has been updated successfully.


### MobileApp section

Create requirements for python to exec code properly

In [1]:
%%writefile ./MobileApp/requirements.txt

Flask

Writing ./MobileApp/requirements.txt


Generate app dockerfile for MobileAPP

In [2]:
%%writefile ./MobileApp/app.dockerfile

FROM python:3.12

# Set the working directory inside the container to /app
WORKDIR /app

# Copy the requirements.txt file into the container at /app
COPY requirements.txt .

# Install the Python dependencies specified in requirements.txt
RUN pip install --no-cache-dir -r requirements.txt

# Copy the application code into the container at /app
COPY ./source_code/ ./source_code/

# Expose port 80 to allow external access to the container's port 80
EXPOSE 80

# Specify the command to run when the container starts
CMD ["python", "./source_code/API_korisnik/app.py"]

Writing ./MobileApp/app.dockerfile


Modify docker compose so that it adds second part of app as service

In [4]:
 
from ruamel.yaml import YAML

# Create a YAML object
yaml = YAML()
yaml.preserve_quotes = True  # Preserves quotes in the YAML file
yaml.indent(mapping = 2, sequence = 2, offset = 2)

#Setup file edit path 
docker_compose_path = './docker-compose.yml'  

# Read the docker-compose.yml file
with open(docker_compose_path, 'r') as file:
    docker_compose = yaml.load(file)

if docker_compose['services'] is None:
    docker_compose['services']={}

if docker_compose['volumes'] is None:
    docker_compose['volumes']={}

MobileAPP = {
    'container_name': 'mobile_app', #TODO: important for nginx detection
    'build': {
        'context': './MobileApp',
        'dockerfile': 'app.dockerfile'
    },
    'networks': [
        'app-network'
    ],



}

#Add the ManagerApp service to the docker-compose.yml file
docker_compose['services']['mobile_app'] = MobileAPP

# Write the updated configuration back to docker-compose.yml
with open(docker_compose_path, 'w') as file:
    yaml.dump(docker_compose, file)

print("\ndocker-compose.yml has been updated successfully.")
 


docker-compose.yml has been updated successfully.
